In [1]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }

quiet_library(hise)
quiet_library(DESeq2)
quiet_library(dplyr)
quiet_library(purrr)

## Filtering functions

In [2]:
select_pseudobulk_samples <- function(pb_data, ...) {
    pb_data$sample_meta <- pb_data$sample_meta %>%
      filter( ... )

    pb_data$agg_mat <- pb_data$agg_mat[, pb_data$sample_meta$barcodes]
    pb_data$norm_mat <- pb_data$norm_mat[, pb_data$sample_meta$barcodes]
    pb_data$detect_mat <- pb_data$detect_mat[, pb_data$sample_meta$barcodes]

    pb_data
}

**Note**: Select samples first, then select genes to ensure cutoffs are applied to only the samples being used for your test.

In [24]:
select_pseudobulk_genes <- function(pb_data, detect_cutoff = 0.1) {
    detect_totals <- rowSums(pb_data$detect_mat)
    detect_fracs <- detect_totals / sum(pb_data$sample_meta$n_cells)
    keep_genes <- rownames(pb_data$detect_mat)[detect_fracs > detect_cutoff]
    
    pb_data$agg_mat <- pb_data$agg_mat[keep_genes,]
    pb_data$norm_mat <- pb_data$norm_mat[keep_genes,]
    pb_data$detect_mat <- pb_data$detect_mat[keep_genes,]

    pb_data
}

## Conversion to DESeqDataSet format

In [33]:
pseudobulk_to_deseq2 <- function(pb_data, type = c("raw", "norm"), design) {
    if(type == "raw") {
        mat <- pb_data$agg_mat
    } else if (type == "norm") {
        mat <- pb_data$norm_mat
    }
    
    sample_meta <- as.data.frame(pb_data$sample_meta)
    rownames(sample_meta) <- sample_meta$barcodes
    
    DESeqDataSetFromMatrix(
        mat, 
        colData = sample_meta,
        design = {{ design }}
    )
}

## Extract results with nice structure

In [65]:
format_deseq_results <- function(dds, cell_type, group_by, fg, bg) {
    meta <- as.data.frame(colData(dds))
    n_fg <- sum(meta[[group_by]] == fg)
    n_bg <- sum(meta[[group_by]] == bg)
    
    res <- results(dds, contrast = c(group_by, fg, bg))
    res <- data.frame(res)
    
    res <- res %>%
      arrange(padj) %>%
      mutate(direction = ifelse(log2FoldChange > 0,
                                paste0("HigherIn", fg), 
                                paste0("HigherIn", bg))) %>%
      mutate(gene = rownames(.),
             cell_type = cell_type,
             fg = fg, n_fg = n_fg,
             bg = bg, n_bg = n_bg) %>%
      select(cell_type, fg, n_fg, bg, n_bg, gene, log2FoldChange, padj, direction,
             baseMean, lfcSE, stat, pvalue)

    res
    
}

## Example case 1

Test CMV status in Adaptive NK cells from BR2 Female samples

Select samples

In [68]:
pb_data <- readRDS("pseudobulk_l3/diha_Adaptive-NK-cell_pseudobulk.rds")

In [69]:
dim(pb_data$agg_mat)

[1] 33538   865

In [70]:
cell_type <- "Adaptive NK cell"

pb_data <- select_pseudobulk_samples(
    pb_data,
    cohort.cohortGuid == "BR2",
    subject.biologicalSex == "Female"
)

In [71]:
dim(pb_data$agg_mat)

[1] 33538   253

Select genes

In [72]:
pb_data <- select_pseudobulk_genes(
    pb_data,
    detect_cutoff = 0.05
)

In [73]:
dim(pb_data$agg_mat)

[1] 6618  253

Convert to DESeq2 object with design

In [74]:
deseq_data <- pseudobulk_to_deseq2(
    pb_data, 
    type = "raw", 
    design = ~ subject.cmv
)

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


Run DESeq2

In [75]:
deseq_output <- DESeq(deseq_data)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



Structure results

In [76]:
deseq_results <- format_deseq_results(
    deseq_output,
    cell_type,
    group_by = "subject.cmv",
    fg = "Positive", bg = "Negative"
)

In [77]:
head(deseq_results)

,cell_type,fg,n_fg,bg,n_bg,gene,log2FoldChange,padj,direction,baseMean,lfcSE,stat,pvalue
,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
SELL,Adaptive NK cell,Positive,163,Negative,90,SELL,-1.1911208,1.627572e-21,HigherInNegative,23.102969,0.11454300,-10.398897,2.508201e-25
GZMH,Adaptive NK cell,Positive,163,Negative,90,GZMH,0.4654324,2.843111e-20,HigherInPositive,146.065834,0.04629024,10.054654,8.762864e-24
FCER1G,Adaptive NK cell,Positive,163,Negative,90,FCER1G,-0.6172658,7.731360e-19,HigherInNegative,83.946718,0.06374988,-9.682620,3.574369e-22
MTRNR2L8,Adaptive NK cell,Positive,163,Negative,90,MTRNR2L8,-1.7664710,6.453594e-15,HigherInNegative,4.673367,0.20352426,-8.679413,3.978175e-18
CEBPD,Adaptive NK cell,Positive,163,Negative,90,CEBPD,-0.7747188,7.036123e-15,HigherInNegative,15.996495,0.08984031,-8.623287,6.505893e-18
CD3D,Adaptive NK cell,Positive,163,Negative,90,CD3D,1.4021423,7.036123e-15,HigherInPositive,22.010604,0.16228296,8.640108,5.615977e-18


## Example case 2

Test Flu response in Plasma cells from BR1 CMV+ samples

Select samples

In [78]:
pb_data <- readRDS("pseudobulk_l3/diha_Plasma-cell_pseudobulk.rds")

In [79]:
dim(pb_data$agg_mat)

[1] 33538   850

In [80]:
cell_type <- "Plasma cell"

pb_data <- select_pseudobulk_samples(
    pb_data,
    cohort.cohortGuid == "BR1",
    subject.cmv == "Positive",
    sample.visitName %in% c("Flu Year 1 Day 0", "Flu Year 1 Day 7")
)

In [81]:
dim(pb_data$agg_mat)

[1] 33538    35

Select genes

In [82]:
pb_data <- select_pseudobulk_genes(
    pb_data,
    detect_cutoff = 0.1
)

In [83]:
dim(pb_data$agg_mat)

[1] 7725   35

Convert to DESeq2 object with design

In [84]:
deseq_data <- pseudobulk_to_deseq2(
    pb_data, 
    type = "raw", 
    design = ~ sample.visitName
)

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]



Run DESeq2

In [85]:
deseq_output <- DESeq(deseq_data)

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

final dispersion estimates

fitting model and testing

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  fo

Structure results

In [86]:
deseq_results <- format_deseq_results(
    deseq_output,
    cell_type,
    group_by = "sample.visitName",
    fg = "Flu Year 1 Day 7", bg = "Flu Year 1 Day 0"
)

In [87]:
head(deseq_results)

,cell_type,fg,n_fg,bg,n_bg,gene,log2FoldChange,padj,direction,baseMean,lfcSE,stat,pvalue
,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
ASB2,Plasma cell,Flu Year 1 Day 7,18,Flu Year 1 Day 0,17,ASB2,-1.2864758,0.2391302,HigherInFlu Year 1 Day 0,3.060223,0.3176596,-4.049856,5.124907e-05
LTK,Plasma cell,Flu Year 1 Day 7,18,Flu Year 1 Day 0,17,LTK,-1.0960718,0.2391302,HigherInFlu Year 1 Day 0,6.633488,0.2736480,-4.005407,6.191072e-05
EXOSC9,Plasma cell,Flu Year 1 Day 7,18,Flu Year 1 Day 0,17,EXOSC9,-0.7760153,0.7727507,HigherInFlu Year 1 Day 0,5.270538,0.2205191,-3.519038,4.331139e-04
RAD21,Plasma cell,Flu Year 1 Day 7,18,Flu Year 1 Day 0,17,RAD21,-0.5093242,0.7727507,HigherInFlu Year 1 Day 0,19.824227,0.1463294,-3.480669,5.001623e-04
B4GALT1,Plasma cell,Flu Year 1 Day 7,18,Flu Year 1 Day 0,17,B4GALT1,1.4582484,0.7727507,HigherInFlu Year 1 Day 7,24.338660,0.4043013,3.606836,3.099534e-04
MANBA,Plasma cell,Flu Year 1 Day 7,18,Flu Year 1 Day 0,17,MANBA,-1.2467109,0.7916603,HigherInFlu Year 1 Day 0,2.318551,0.3640071,-3.424964,6.148818e-04


In [88]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.6 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] purrr_1.0.2                 dplyr_1.1.4                
 [3] DESeq2_1.42.0               SummarizedExperiment_1.32.0
 [5] Biobase_2.62.0              MatrixGenerics_1.14.0      
 [7] matrixStats_1.2.0           GenomicRanges_1.54.1       
 [9] GenomeInfoDb_1.38.5         IRanges_2.36.0        